In [ ]:
class Draw_customer_field():
    def __init__(self) -> None:
        pass

    def __call__(self):
        pass

In [1]:
from fakedata import FakeData
from PIL import Image, ImageDraw, ImageFont
from invoice_generator import LayoutManager, Draw_Table

class InvoiceGenerator():
    def __init__(self, canvas, num_documents = 1, det = True) -> None:
        self.canvas = canvas
        self.drawer = ImageDraw.Draw(self.canvas)
        self.font = ImageFont.truetype("arial.ttf", 30)
        self.num_documents = num_documents
        self.det = det
        self.labels = []     

    def __call__(self):
        # 1. Generate the layout
        layout_manager = LayoutManager()
        self.layout = layout_manager()
        # self.visualize_layout()

        # 2. Generate the fake data
        fake_data_generator = FakeData()
        fake_data = fake_data_generator(self.num_documents, self.det)

        # 3. Draw the fake data on the image
        table_drawer = Draw_Table()
        for document in fake_data:
            # Draw the table on the document
            self.labels = table_drawer(self.labels, self.canvas, self.layout['T_field'], document['I_Currency'])
        print(self.labels)
        self.canvas.show()

    def visualize_layout(self):
        for key in self.layout:
            self.drawer.rectangle(self.layout[key], outline='black')
            self.drawer.text(self.layout[key], key, font=self.font, fill='black')
        self.canvas.show()

blank_dir = r'blank.tif'
blank = Image.open(blank_dir)

invoice_generator = InvoiceGenerator(blank, 1, True)
invoice_generator()

[{'text': 'Description', 'label': 'Other', 'box': [112, 989, 264, 1017], 'words': [{'box': (112, 989, 264, 1017), 'text': 'Description'}]}, {'text': 'Quantity', 'label': 'Other', 'box': [694, 989, 806, 1017], 'words': [{'box': (694, 989, 806, 1017), 'text': 'Quantity'}]}, {'text': 'Tax %', 'label': 'Other', 'box': [1080, 989, 1165, 1011], 'words': [{'box': (1080, 989, 1130, 1011), 'text': 'Tax'}, {'box': (1138, 989, 1165, 1011), 'text': '%'}]}, {'text': 'Brutto', 'label': 'Other', 'box': [1444, 989, 1524, 1011], 'words': [{'box': (1444, 989, 1524, 1011), 'text': 'Brutto'}]}, {'text': 'Professional software', 'label': 'Other', 'box': [112, 1063, 401, 1085], 'words': [{'box': (112, 1063, 279, 1085), 'text': 'Professional'}, {'box': (287, 1063, 401, 1085), 'text': 'software'}]}, {'text': '4', 'label': 'Other', 'box': [789, 1063, 806, 1085], 'words': [{'box': (789, 1063, 806, 1085), 'text': '4'}]}, {'text': '9', 'label': 'Other', 'box': [1148, 1063, 1165, 1085], 'words': [{'box': (1148, 10

In [ ]:
import json

with open('annotation.json', 'w') as fp:
    json.dump(label, fp)